# Regresión lineal simple

Es un método que busca modelar la relación entre una varieble independiente **`x`** y una variable dependiente **`y`**, de un conjunto de **`m`** datos conocidos, a travéz de una función líneal:

$$ f(x_{i}) \;\; = \;\; w * x_{i} + b \;\; = \;\; \hat{y_{i}} \;\;\;\;\; \to \;\;\;\;\; i = 1, ..., m $$

In [1]:
def linear_function(x: float, w: float, b: float) -> float:
    return w * x + b

De esta forma podremos realizar una predicción de los valores de **`y`** para nuevos valores de **`x`**.

![Datos y función lineal modelada](images/introduction.png)

El objetivo es encontrar los parametros **`w`** (peso o weight en inglés) y **`b`** (parcialidad o bias en inglés), que formen la función lineal que **mejor se adapte a los datos**, a partir de los mismos. Pero primero necesitamos una forma de cuantificar que tan bien se adapta, o explica los datos, una función lineal cualquiera.

## Función de costo

Para darle un valor al ajuste de una recta con respecto a ciertos datos, vamos a construir una linea cualquiera y graficarla junto al conjunto de datos que queremos modelar.

![Datos y función lineal de prueba](images/data_and_one_linear_function.png)

In [4]:
import csv

# Extract data from file to lists of values
x_values = []
y_values = []
with open("data/notebook_example_data.csv", "r") as file:
    reader = csv.DictReader(file)
    for data in reader:
        x_values.append(float(data["x"]))
        y_values.append(float(data["y"]))

w = 0.35
b = 4

print("x values: {}".format(x_values))
print("y values: {}".format(y_values))
print("Weight value: {}".format(w))
print("Bias value: {}".format(b))

x values: [1.0, 1.9, 2.8, 3.7, 4.6, 5.5, 6.4, 7.3, 8.2, 9.1]
y values: [1.73700352294, 3.54427728722, 1.79724195301, 2.59844106704, 4.785076322, 5.30633212332, 4.15850757058, 4.36079383376, 8.08863005356, 6.55052484748]
Weight value: 0.35
Bias value: 4


Ahora vamos a definir los errores $ \varepsilon_{i} $ como la diferencia entre el valor real $ y_{i} $ y el valor de la linea $ \hat{y_{i}} $ para cada $ x_{i} $ del conjunto de datos.

$$ \varepsilon_{i} \;\; = \;\; y_{i} - \hat{y_{i}} \;\; = \;\; y_{i} - (w * x_{i} + b) \;\;\;\;\; \to \;\;\;\;\; i = 1, ..., m $$

![Datos y errores](images/data_line_and_errors.png)



In [14]:
def error_function(y: float, prediction: float) -> float:
    return y - prediction

errors = [error_function(y, linear_function(x, w, b)) for y, x in zip(y_values, x_values)]
print("Errors of the line with respect to the data: {}".format(errors))

Errors of the line with respect to the data: [-2.6129964770599994, -1.1207227127800001, -3.1827580469899996, -2.69655893296, -0.8249236779999993, -0.61866787668, -2.08149242942, -2.19420616624, 1.21863005356, -0.6344751525199994]


Estos errores nos pueden dar una intuición de qué tan bien se ajusta una recta a un conjunto de datos, por ejemplo si sumamos a todos. Rectas que se adapten mejor tendrán una suma de errores menor, y rectas que se adapten peor tendán una suma de errores mayor. Pero como se puede observar en la figura y los cálculos, los errores pueden ser positivos o negativos, lo cual hace que la suma no refleje realmente la distancia entre la recta y los datos.
Para mejorar nuestra forma de valorizar el ajuste de una recta, vamos a definir la **función de pérdida** (loss function en inglés) como la mitad del error al cuadrado.

$$ \ell_{i} \;\; = \;\; \frac{1}{2} * ( y_{i} - \hat{y_{i}} )^2 \;\; = \;\; \frac{1}{2} * ( \varepsilon_{i} )^2 \;\;\;\;\; \to \;\;\;\;\; i = 1, ..., m $$

Ventajas:
 - Al elevar el error al cuadrado, me aseguro que son todos positivos, y contribuirán a la suma de errores siempre aumentandola.
 - Al elevar el error al cuadrado tabién estoy haciendo que los errores más grandes incidán todavía más sobre la suma de errores, mientras que errores más chicos contribuirán menos al error total.
 - El echo de usar la mitad del error al cuadrado servirá exclusivamente a la hora de derivar la función para hacer dicho cálculo más sencillo.
 


In [15]:
def loss_function(y: float, prediction: float) -> float:
    return 0.5 * (error_function(y, prediction) ** 2)

losses = [loss_function(y, linear_function(x, w, b)) for y, x in zip(y_values, x_values)]
print("Losses of the line with respect to the data: {}".format(losses))

Losses of the line with respect to the data: [3.413875294563984, 0.6280096994704814, 5.064974392839798, 3.6357150394631867, 0.34024953726252327, 0.19137497081786986, 2.1663053668663865, 2.4072703499828187, 0.7425296037198243, 0.20127935958263826]


Por último vamos a definir la **función de costo**. Esta puede ser cualquier función, siempre y cuando represente la * **distancia** * de la recta con respecto a los datos, es decir, que cuanto menor sea el costo, mejor va a ser la recta para explicar el comportamiento de la variable dependiente **`y`** con respecto a la variable independiente **`x`** de dicho conjunto de datos. En este caso vamos a utilizar el promedio de las **pérdidas**.

$$ C(y_{1},\;...\;,y_{m}, \hat{y_{1}},\;...\;,\hat{y_{m}}) \;\; = \;\; \frac{1}{m} * \displaystyle\sum_{i=1}^{m} \frac{1}{2} * ( y_{i} - \hat{y_{i}} )^2 $$

Esta función nos permite saber que tan buenos son lo parametros **w** y **b** para modelar los datos.

In [11]:
from typing import List
def cost_function(x_values: List[float], y_values: List[float], w: float, b: float) -> float:
    losses = [loss_function(y, linear_function(x, w, b)) for y, x in zip(y_values, x_values)]
    
    return sum(losses) / len(losses)

print(cost_function(x_values, y_values, w, b))

1.8791583614569511


Calculamos el costo de nuestra recta con respecto a los datos, pero este número por si solo no nos dice nada. Lo podríamos utilizar para compararlo con el costo de otra recta y ver cuál es mejor, pero también lo podemos utilizar para tener una intuición con respecto a la recta actual: de lo que estamos seguros, es de que si cambiando los parametros **w** y **b**, logramos bajar el costo, la recta se va a acercar a los datos.

Esto es exactamente lo que quiero lograr con la regresión lineal: ajustar los parametros de la función lineal, hasta que la misma sea lo mejor posible para modelar la dependencia entre la variable **`x`** y la variable **`y`**.

## Ajustando la recta a los datos (gradiente descendente)

Continuando con los datos y la funcion linear que ya teníamos, vamos a tratar de cambiar los valores de **w** y de **b** para que la función lineal se ajuste mejor a los datos, es decir, que queremos modificar los parametros de la recta, de forma de que baje el costo de la misma. Para esto vamos a reescribir a la función de costo, como:

$$ C(y_{1},\;...\;,y_{m}, x_{i},\;...\;,x_{m}, w, b) \;\; = \;\; \frac{1}{m} * \displaystyle\sum_{i=1}^{m} \frac{1}{2} * ( y_{i} - (w * x_{i} + b) )^2 $$

Vemos que la función de costo depende de diversas variables, entre las cuales se encuentran **w** y **b**. Entonces podemos obtener algebraícamente y graficar la función de costo cuando todas las variables toman los valores del problema (los **x** e **y** de los datos y el **b** de la recta) y **w** es la única variable.

Trabajando la función de costo se puede llegar a la expresión:

$$ C(y_{1},\;...\;,y_{m}, x_{i},\;...\;,x_{m}, w, b) \;\; = \;\; \frac{1}{2 * m} * \bigg( w^2 * \displaystyle\sum_{i=1}^{m} \Big[ x_{i}^2 \Big] \; + \; w * \displaystyle\sum_{i=1}^{m} \Big[ 2 * x_{i} * (b-y_{i}) \Big] \; + \; \displaystyle\sum_{i=1}^{m} \Big[ y_{i}^2 - 2*y_{i}*b+b^2 \Big] \bigg)  $$

Si reemplazamos por los valores de los datos del problema, y por nuestro actual valor de **b**, tenemos:

$$  C(w) \;\; = \;\; \frac{1}{2 * 10} * \bigg( w^2 * 321.85 + w * (-113.3972) + 37.8456 \bigg) $$


$$ C(w\;=\;0.35) \;\; = \;\; 1.879 $$


Se puede comprobar que para el valor de **w** de nuestra recta (0.35), el costo sigue dando el mismo resultado, ya que esta función no fue modificada, solo reorganizada.

Podemos hacer lo mismo, pero dejando a **b** como única variable, y utilizando el **w** actual:

$$ C(y_{1},\;...\;,y_{m}, x_{i},\;...\;,x_{m}, w, b) \;\; = \;\; \frac{1}{2 * m} * \bigg( b^2 * m + b * \displaystyle\sum_{i=1}^{m} \Big[ 2 * (w * x_{i} - y_{i}) \Big] + \displaystyle\sum_{i=1}^{m} \Big[ y_{i}^2 - 2 * y_{i} * x_{i} * w + w^2 * x_{i}^2 \Big] \bigg) $$


$$ C(b) \;\; = \;\; \frac{1}{2 * 10} * \bigg( b^2 * 10 + b * (-50.5036) + 79.5978 \bigg) $$


$$ C(b\;=\;4) \;\; = \;\; 1.879 $$


Nuevamente vemos que para el valor de **b** de nuestra recta (4), el costo sigue dando el mismo resultado.

Ahora podemos graficar nuestras funciones de costo con respecto a **w** y **b** respectivamente.

![Costo con respecto al w y al b](images/cost_weight_and_bias.png)

Mirando las gráficas, es fácil ver que para bajar el costo de nuestro modelo, debemos reducir los valores de **w** y de **b**. Pero no necesariamente tenemos que graficar estas funciones para obtener esta información.
La [derivada parcial](https://es.wikipedia.org/wiki/Derivada_parcial) de una función con respecto a una de sus variables nos dice la rapidez con la que cambia dicha función para cada posible valor de la variable. Entonces si obtenemos la derivada de la función de costo con respecto a **w** y **b** podemos saber, para cada posible valor de las variables, . 

https://mickteaching.wordpress.com/2016/04/19/data-need-to-be-normally-distributed-and-other-myths-of-linear-regression/
https://stats.stackexchange.com/questions/148803/how-does-linear-regression-use-the-normal-distribution
https://en.wikipedia.org/wiki/Simple_linear_regression
https://www.researchgate.net/post/Is_linear_regression_valid_when_the_outcome_dependant_variable_not_normally_distributed
https://es.wikipedia.org/wiki/Derivada_parcial
https://en.wikipedia.org/wiki/Cost_function

### Linea de mejor *ajuste*

Explicar porque el echo de que los errores con respecto a la linea esten normalmente distribuídos hace que sea un buen modelo para predecir el mundo.


El objetivo es encontrar un valor para los parametros **`w`** y **`b`** que hagan que la función lineal se *ajuste* lo mejor posible a los datos. Esto significa que los errores de cada dato conocido, con respecto a la predicción efectuada por la función, se encuentren en una distribución normal con media 0 ($ \mu = 0 $).

![Datos y errores](images/data_and_errors.png)